In [2]:
import pandas as pd
import seaborn as sns
import numpy as np
from scipy.optimize import minimize
from scipy.optimize import minimize_scalar
%matplotlib inline
import matplotlib.pyplot as plt

#import models
from sklearn.linear_model import LinearRegression
from sklearn.neural_network import MLPRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn import linear_model
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
#from xgboost import XGBRegressor

from sklearn import model_selection
from sklearn.metrics import explained_variance_score
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import mean_squared_error
from tabulate import _table_formats, tabulate
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import train_test_split

from sklearn import metrics

In [3]:
datas = './MyData.xlsx'

In [5]:
pd.set_option("display.max_columns", None)
df=pd.read_excel(datas)
df.head(2)
df['logD'] = np.log10(df['diffusivity'])
# df = df.dropna()
# df[df.gas == 'propylene']['logD'].max()
df[df.gas == 'H2']['logD'].count()

61

In [4]:
# df = df[df.gas != 'butane']
# df = df[df.gas != 'ch4']
# df = df[df.type != 'Co-Br-ZIF-7-8']
# df.type.unique

# Keep only the columns I need

In [6]:
# df2=df[[ 'aperture', 'size - van der Waals (Å)','mass', 'ascentricF', 'logD', 'size - kinetic diameter (Å)', 'gas', 'apertureAtom_e', 'ionicRad']]
df2=df[[ 'aperture', 'size - van der Waals (Å)','mass', 'ascentricF', 'logD', 'size - kinetic diameter (Å)', 'ionicRad', 
       'Μ-N_lff', 'Μ-N_kFF', 'ionicRad', 'MetalCharge',
       'apertureAtom_σ', 'apertureAtom_e', 'linker_length1', 'linker_length2',
       'linker_length3', 'linker_mass1', 'linker_mass2', 'linker_mass3',
       'func1_length', 'func2_length', 'func3_length', 'func1_mass',
       'func2_mass', 'func3_mass', 'func1_charge', 'func2_charge',
       'func3_charge',]]
df2.head()

,aperture,size - van der Waals (Å),mass,ascentricF,logD,size - kinetic diameter (Å),ionicRad,Μ-N_lff,Μ-N_kFF,ionicRad,MetalCharge,apertureAtom_σ,apertureAtom_e,linker_length1,linker_length2,linker_length3,linker_mass1,linker_mass2,linker_mass3,func1_length,func2_length,func3_length,func1_mass,func2_mass,func3_mass,func1_charge,func2_charge,func3_charge
0,3.92,2.66,4.002,-0.390,-7.221849,2.60,92,2.257,45354.6,92,1.1901,0.25,0.0627,4.438,4.438,4.438,81.0,81.0,81.0,3.78,3.78,3.78,15.0,15.0,15.0,-0.1001,-0.1001,-0.1001
1,3.92,2.76,2.010,-0.217,-7.301030,2.89,92,2.257,45354.6,92,1.1901,0.25,0.0627,4.438,4.438,4.438,81.0,81.0,81.0,3.78,3.78,3.78,15.0,15.0,15.0,-0.1001,-0.1001,-0.1001
2,3.92,2.94,31.999,0.022,-8.000000,3.46,92,2.257,45354.6,92,1.1901,0.25,0.0627,4.438,4.438,4.438,81.0,81.0,81.0,3.78,3.78,3.78,15.0,15.0,15.0,-0.1001,-0.1001,-0.1001
3,3.92,3.24,44.010,0.225,-8.397940,3.30,92,2.257,45354.6,92,1.1901,0.25,0.0627,4.438,4.438,4.438,81.0,81.0,81.0,3.78,3.78,3.78,15.0,15.0,15.0,-0.1001,-0.1001,-0.1001
4,3.92,3.13,28.000,0.037,-8.301030,3.64,92,2.257,45354.6,92,1.1901,0.25,0.0627,4.438,4.438,4.438,81.0,81.0,81.0,3.78,3.78,3.78,15.0,15.0,15.0,-0.1001,-0.1001,-0.1001


# Get a first correlation of ionic radius with aperture

In [7]:
df2[['aperture','logD']].corr(method="spearman")

,aperture,logD
aperture,1.00000,0.59506
logD,0.59506,1.00000


In [8]:
df2=df2.rename(columns={'size - van der Waals (Å)':'diameter', 'size - kinetic diameter (Å)':'kdiameter', 'apertureAtom_e':'e' })

In [9]:
df2 = df2.dropna()
df2=df2.reset_index(drop=True)
df2

,aperture,diameter,mass,ascentricF,logD,kdiameter,ionicRad,Μ-N_lff,Μ-N_kFF,ionicRad,MetalCharge,apertureAtom_σ,e,linker_length1,linker_length2,linker_length3,linker_mass1,linker_mass2,linker_mass3,func1_length,func2_length,func3_length,func1_mass,func2_mass,func3_mass,func1_charge,func2_charge,func3_charge
0,3.92,2.66,4.002,-0.390,-7.221849,2.60,92,2.257,45354.60,92,1.1901,0.25,0.0627,4.438,4.438,4.438,81.000,81.000,81.000,3.78,3.78,3.78,15.0,15.0,15.0,-0.1001,-0.1001,-0.1001
1,3.92,2.76,2.010,-0.217,-7.301030,2.89,92,2.257,45354.60,92,1.1901,0.25,0.0627,4.438,4.438,4.438,81.000,81.000,81.000,3.78,3.78,3.78,15.0,15.0,15.0,-0.1001,-0.1001,-0.1001
2,3.92,2.94,31.999,0.022,-8.000000,3.46,92,2.257,45354.60,92,1.1901,0.25,0.0627,4.438,4.438,4.438,81.000,81.000,81.000,3.78,3.78,3.78,15.0,15.0,15.0,-0.1001,-0.1001,-0.1001
3,3.92,3.24,44.010,0.225,-8.397940,3.30,92,2.257,45354.60,92,1.1901,0.25,0.0627,4.438,4.438,4.438,81.000,81.000,81.000,3.78,3.78,3.78,15.0,15.0,15.0,-0.1001,-0.1001,-0.1001
4,3.92,3.13,28.000,0.037,-8.301030,3.64,92,2.257,45354.60,92,1.1901,0.25,0.0627,4.438,4.438,4.438,81.000,81.000,81.000,3.78,3.78,3.78,15.0,15.0,15.0,-0.1001,-0.1001,-0.1001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
590,2.95,2.76,2.010,-0.217,-7.455932,2.89,71,2.139,38994.88,71,1.9871,0.34,1.2552,6.410,6.410,6.410,134.906,134.906,134.906,3.78,3.78,3.78,15.0,15.0,15.0,-0.2710,-0.2710,-0.2710
591,2.95,2.94,31.999,0.022,-9.180456,3.46,71,2.139,38994.88,71,1.9871,0.34,1.2552,6.410,6.410,6.410,134.906,134.906,134.906,3.78,3.78,3.78,15.0,15.0,15.0,-0.2710,-0.2710,-0.2710
592,2.95,3.24,44.010,0.225,-9.096910,3.30,71,2.139,38994.88,71,1.9871,0.34,1.2552,6.410,6.410,6.410,134.906,134.906,134.906,3.78,3.78,3.78,15.0,15.0,15.0,-0.2710,-0.2710,-0.2710
593,2.95,3.13,28.000,0.037,-9.638272,3.64,71,2.139,38994.88,71,1.9871,0.34,1.2552,6.410,6.410,6.410,134.906,134.906,134.906,3.78,3.78,3.78,15.0,15.0,15.0,-0.2710,-0.2710,-0.2710


# Use only van der Waals

In [10]:
# x_ar = np.asanyarray(df2[['aperture', 'diameter']])
# # x_ar = np.asanyarray(df2[['aperture', 'diameter','mass', 'ascentricF', 'kdiameter', 'ionicRad']])
# y_ar = np.asanyarray(df2[['logD']])

x_ar = np.array(df2[['aperture', 'diameter']])
# x_ar = np.array(df2[['aperture']])
# x_ar = np.asanyarray(df2[['aperture', 'diameter','mass', 'ascentricF', 'kdiameter', 'ionicRad']])
y_ar = np.array(df2[['logD']])

x1 = (df2[['aperture', 'diameter']])
# x1 = (df2[['aperture']])
y1 = (df2[['logD']])

In [11]:
x_ar.shape
x_ar.ndim
x_ar

array([[3.92, 2.66],
       [3.92, 2.76],
       [3.92, 2.94],
       ...,
       [2.95, 3.24],
       [2.95, 3.13],
       [2.95, 3.25]])

In [12]:
data = x_ar
len(data.shape)

2

## Optimizer

In [61]:
regr = linear_model.LinearRegression()
tree=DecisionTreeRegressor(max_depth=5,random_state=1)
print(regr)

LinearRegression()


In [46]:
def fitness(X, *args): 
    if X[0] < 5:
        return 7
    else:
        return -1.0/X[0]+ X[1]
bounds=[(-5,100), (-50, 50)]

In [49]:
res = minimize(fitness, np.array([1.0, 5.0]), bounds=bounds, method='L-BFGS-B')

In [50]:
print (str(res))


      fun: 7
 hess_inv: <2x2 LbfgsInvHessProduct with dtype=float64>
      jac: array([0., 0.])
  message: b'CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL'
     nfev: 3
      nit: 0
   status: 0
  success: True
        x: array([1., 5.])


In [69]:
x_train, x_test, y_train, y_test = train_test_split(x_ar, y_ar, test_size=0.3, random_state=None)
# x_train= x_train.reshape(-1, 1)
# y_train= y_train.reshape(-1, 1)
# x_test = x_test.reshape(-1, 1)
regression=regr.fit(x_train, y_train)
regression.predict([[1.0, 2.0]])

[[-17.2]]

array([[-15.28355583]])

In [78]:
initialParameterValues = x_ar[0]
print(initialParameterValues)
bounds = [(np.min(x_ar[:, 0]), np.max(x_ar[:, 0])), (np.min(x_ar[:, 1]), np.max(x_ar[:, 1]))]
print(str(bounds))

regression.predict([initialParameterValues])
def fitness(PARAMS, *args):
    floatResult = regression.predict([PARAMS])[0][0]
    return abs(floatResult - -12.0)

res = minimize(fitness, initialParameterValues, bounds=bounds, method='L-BFGS-B')
print(str(res))

[3.92 2.66]
[(1.96, 3.92), (2.66, 4.8)]
      fun: 1.1022102341939899e-08
 hess_inv: <2x2 LbfgsInvHessProduct with dtype=float64>
      jac: array([3.82587331, 6.94597801])
  message: b'ABNORMAL_TERMINATION_IN_LNSRCH'
     nfev: 201
      nit: 4
   status: 2
  success: False
        x: array([3.20457812, 3.44112388])


In [17]:
myX=x_ar.reshape(-1,1)
myY=y_ar.reshape(-1,1)

# x_train, x_test, y_train, y_test = train_test_split(myX, myY, test_size=0.3, random_state=None)
# # x_train= x_train.reshape(-1, 1)
# # y_train= y_train.reshape(-1, 1)
# # x_test = x_test.reshape(-1, 1)
# regression=regr.fit(x_train, y_train)
x_ar

array([[3.92, 2.66],
       [3.92, 2.76],
       [3.92, 2.94],
       ...,
       [2.95, 3.24],
       [2.95, 3.13],
       [2.95, 3.25]])

In [16]:
# res = minimize(model, myX, method='Nelder-Mead',
#                options={'xatol': 1e-8, 'disp': True})

# res = minimize_scalar(model, bounds=x_ar, method='bounded')
# res = minimize(model, x_ar, method='L-BFGS-B')

# a = np.array([[0.01, 0.2, 0.4],[0.01, 0.2, 0.4]])
a =[[0.01, 0.2, 0.4],[0.01, 0.2, 0.4]]

res = minimize(model, x_ar, method='L-BFGS-B')

ValueError: Expected 2D array, got 1D array instead:
array=[3.92 2.66 3.92 ... 3.13 2.95 3.25].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.

In [195]:
x_ar

array([[3.92, 2.66],
       [3.92, 2.76],
       [3.92, 2.94],
       ...,
       [2.95, 3.24],
       [2.95, 3.13],
       [2.95, 3.25]])